# 01 — Getting Started: Your First Vault

This notebook introduces the Force Density Method (FDM) for finding compression-only vault shapes.

**The key insight:** If you hang a net from its edges and let gravity pull it down, the shape it takes is pure tension. Flip it upside down and you get a pure compression vault. This is how Gaudi designed the Sagrada Familia — with hanging chain models.

The FDM lets us do this computationally. We define a mesh, assign force densities to the edges, apply loads, and solve for the equilibrium shape.

In [ ]:
from compas.datastructures import Mesh
from compas_fd.solvers import fd_numpy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

## Step 1: Create the Ground Mesh

We start with a flat rectangular grid. This represents the floor plan — the footprint of our vault.

`Mesh.from_meshgrid(dx, nx)` creates a grid of `nx × ny` quad faces spanning `dx × dy` units.

In [ ]:
# Create a 10m × 10m floor plan with a 10×10 grid
mesh = Mesh.from_meshgrid(dx=10.0, nx=10)

print(f"Vertices: {mesh.number_of_vertices()}")
print(f"Edges:    {mesh.number_of_edges()}")
print(f"Faces:    {mesh.number_of_faces()}")

In [ ]:
# Visualize the flat ground mesh
vertices_xyz = np.array(mesh.vertices_attributes('xyz'))

fig, ax = plt.subplots()
for u, v in mesh.edges():
    p0 = mesh.vertex_attributes(u, 'xyz')
    p1 = mesh.vertex_attributes(v, 'xyz')
    ax.plot([p0[0], p1[0]], [p0[1], p1[1]], 'k-', linewidth=0.5)

# Highlight boundary vertices
boundary = list(mesh.vertices_on_boundary())
interior = [v for v in mesh.vertices() if v not in boundary]

bx = [mesh.vertex_attribute(v, 'x') for v in boundary]
by = [mesh.vertex_attribute(v, 'y') for v in boundary]
ix = [mesh.vertex_attribute(v, 'x') for v in interior]
iy = [mesh.vertex_attribute(v, 'y') for v in interior]

ax.scatter(bx, by, c='red', s=30, zorder=5, label=f'Fixed ({len(boundary)})')
ax.scatter(ix, iy, c='blue', s=15, zorder=5, label=f'Free ({len(interior)})')
ax.set_aspect('equal')
ax.set_title('Floor Plan — Red vertices are fixed supports')
ax.legend()
plt.show()

## Step 2: Set Up the FDM Problem

We need four things:
1. **Vertices** — current XYZ coordinates
2. **Fixed vertices** — boundary vertices that don't move (the supports)
3. **Force densities** — one per edge, controls the "stiffness" of each cable in the hanging net
4. **Loads** — forces applied at each vertex (gravity pulling down)

In [ ]:
# Extract mesh data for the solver
vertices = mesh.vertices_attributes('xyz')
edges = list(mesh.edges())
fixed = list(mesh.vertices_on_boundary())

# Uniform load pulling down (gravity on the hanging net)
load = -1.0
loads = [[0.0, 0.0, load] for _ in vertices]

# Uniform force density on all edges
q = 1.0
forcedensities = [q] * len(edges)

print(f"Free vertices: {len(vertices) - len(fixed)}")
print(f"Fixed vertices: {len(fixed)}")
print(f"Force density: {q}")
print(f"Load per vertex: {load}")

## Step 3: Solve and Invert

The FDM solver finds the equilibrium shape of the hanging net. We then flip it to get the vault.

**Hanging net (tension) → flip Z → vault (compression)**

In [ ]:
# Solve for the hanging equilibrium shape
result = fd_numpy(
    vertices=vertices,
    fixed=fixed,
    edges=edges,
    forcedensities=forcedensities,
    loads=loads,
)

# Flip Z to get the vault (tension → compression)
vault_xyz = result.vertices.copy()
vault_xyz[:, 2] = -vault_xyz[:, 2]

print(f"Peak height: {vault_xyz[:, 2].max():.2f} m")
print(f"Max residual: {np.max(np.linalg.norm(result.residuals, axis=1)):.6f}")

## Step 4: Visualize the Vault

In [ ]:
def plot_vault(xyz, mesh, title="Vault", elev=30, azim=-60, color='steelblue'):
    """Plot a vault surface from vertex coordinates and mesh connectivity."""
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Draw faces as filled polygons
    face_verts = []
    for face in mesh.faces():
        face_vertices = mesh.face_vertices(face)
        polygon = [xyz[v] for v in face_vertices]
        face_verts.append(polygon)
    
    poly = Poly3DCollection(face_verts, alpha=0.7, 
                            facecolor=color, edgecolor='darkslategray', linewidth=0.3)
    ax.add_collection3d(poly)
    
    # Set axis limits
    ax.set_xlim(xyz[:, 0].min() - 0.5, xyz[:, 0].max() + 0.5)
    ax.set_ylim(xyz[:, 1].min() - 0.5, xyz[:, 1].max() + 0.5)
    ax.set_zlim(-0.5, xyz[:, 2].max() + 1.0)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.view_init(elev=elev, azim=azim)
    
    return fig, ax


plot_vault(vault_xyz, mesh, title=f"Compression Vault — q={q}, load={load}")
plt.tight_layout()
plt.show()

## Step 5: Explore Parameters

The shape of the vault depends on two main controls:
- **Force density (q):** Higher q = stiffer net = flatter vault. Lower q = deeper sag = taller vault.
- **Load magnitude:** Heavier load = deeper sag = taller vault.

Let's see what happens when we vary the force density.

In [ ]:
# Compare different force densities
q_values = [0.5, 1.0, 2.0, 5.0]

fig = plt.figure(figsize=(16, 12))

for i, q_val in enumerate(q_values):
    fds = [q_val] * len(edges)
    res = fd_numpy(
        vertices=vertices, fixed=fixed, edges=edges,
        forcedensities=fds, loads=loads,
    )
    
    v_xyz = res.vertices.copy()
    v_xyz[:, 2] = -v_xyz[:, 2]
    
    ax = fig.add_subplot(2, 2, i + 1, projection='3d')
    
    face_verts = []
    for face in mesh.faces():
        fv = mesh.face_vertices(face)
        face_verts.append([v_xyz[v] for v in fv])
    
    poly = Poly3DCollection(face_verts, alpha=0.7,
                            facecolor='steelblue', edgecolor='darkslategray', linewidth=0.3)
    ax.add_collection3d(poly)
    
    ax.set_xlim(-0.5, 10.5)
    ax.set_ylim(-0.5, 10.5)
    ax.set_zlim(-0.5, 15)
    ax.set_title(f'q = {q_val} — height = {v_xyz[:, 2].max():.1f}m', fontweight='bold')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.view_init(elev=25, azim=-60)

fig.suptitle('Force Density Controls Vault Height', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Step 6: Asymmetric Force Densities

So far we've used the same force density on every edge. But we can assign different values to X-direction and Y-direction edges, creating directional vaults (barrel vaults, etc.).

In [ ]:
def directional_forcedensities(mesh, edges, q_x, q_y):
    """Assign different force densities based on edge orientation."""
    fds = []
    for u, v in edges:
        p0 = mesh.vertex_attributes(u, 'xyz')
        p1 = mesh.vertex_attributes(v, 'xyz')
        dx = abs(p1[0] - p0[0])
        dy = abs(p1[1] - p0[1])
        # X-aligned edges get q_x, Y-aligned edges get q_y
        if dx > dy:
            fds.append(q_x)
        else:
            fds.append(q_y)
    return fds


# Barrel vault: stiff in X (q=5), flexible in Y (q=0.5)
fds_barrel = directional_forcedensities(mesh, edges, q_x=5.0, q_y=0.5)

res_barrel = fd_numpy(
    vertices=vertices, fixed=fixed, edges=edges,
    forcedensities=fds_barrel, loads=loads,
)

barrel_xyz = res_barrel.vertices.copy()
barrel_xyz[:, 2] = -barrel_xyz[:, 2]

fig = plt.figure(figsize=(16, 6))

# Symmetric vault
ax1 = fig.add_subplot(121, projection='3d')
fv1 = [[vault_xyz[v] for v in mesh.face_vertices(f)] for f in mesh.faces()]
ax1.add_collection3d(Poly3DCollection(fv1, alpha=0.7, facecolor='steelblue', edgecolor='darkslategray', linewidth=0.3))
ax1.set_xlim(-0.5, 10.5); ax1.set_ylim(-0.5, 10.5); ax1.set_zlim(-0.5, 10)
ax1.set_title('Symmetric (q=1.0 all edges)', fontweight='bold')
ax1.view_init(elev=25, azim=-60)

# Barrel vault
ax2 = fig.add_subplot(122, projection='3d')
fv2 = [[barrel_xyz[v] for v in mesh.face_vertices(f)] for f in mesh.faces()]
ax2.add_collection3d(Poly3DCollection(fv2, alpha=0.7, facecolor='coral', edgecolor='darkslategray', linewidth=0.3))
ax2.set_xlim(-0.5, 10.5); ax2.set_ylim(-0.5, 10.5); ax2.set_zlim(-0.5, 10)
ax2.set_title('Barrel Vault (q_x=5.0, q_y=0.5)', fontweight='bold')
ax2.view_init(elev=25, azim=-60)

plt.tight_layout()
plt.show()

## Step 7: Cross-Section View

Let's look at a cross-section through the middle of the vault to see the profile curve.

In [ ]:
# Extract a cross-section at y ≈ 5.0 (middle of the vault)
mid_y = 5.0
tolerance = 0.01

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (xyz, label, color) in zip(axes, [
    (vault_xyz, 'Symmetric Vault', 'steelblue'),
    (barrel_xyz, 'Barrel Vault', 'coral'),
]):
    mask = np.abs(xyz[:, 1] - mid_y) < tolerance
    section = xyz[mask]
    order = np.argsort(section[:, 0])
    section = section[order]
    
    ax.fill_between(section[:, 0], 0, section[:, 2], alpha=0.3, color=color)
    ax.plot(section[:, 0], section[:, 2], 'o-', color=color, linewidth=2, markersize=5)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Z (m)')
    ax.set_title(f'{label} — Cross Section at Y={mid_y}m', fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.5, max(vault_xyz[:, 2].max(), barrel_xyz[:, 2].max()) + 1)

plt.tight_layout()
plt.show()

## Step 8: Finer Resolution

Let's crank up the mesh resolution for a smoother vault. This is still the same math — just more vertices to work with.

In [ ]:
# High-resolution vault
mesh_hires = Mesh.from_meshgrid(dx=10.0, nx=25)
verts_hr = mesh_hires.vertices_attributes('xyz')
edges_hr = list(mesh_hires.edges())
fixed_hr = list(mesh_hires.vertices_on_boundary())
loads_hr = [[0.0, 0.0, -1.0] for _ in verts_hr]
fds_hr = [1.0] * len(edges_hr)

res_hr = fd_numpy(
    vertices=verts_hr, fixed=fixed_hr, edges=edges_hr,
    forcedensities=fds_hr, loads=loads_hr,
)

hr_xyz = res_hr.vertices.copy()
hr_xyz[:, 2] = -hr_xyz[:, 2]

print(f"High-res: {mesh_hires.number_of_vertices()} vertices, {len(edges_hr)} edges")
print(f"Peak height: {hr_xyz[:, 2].max():.2f} m")

plot_vault(hr_xyz, mesh_hires, 
           title=f"High-Resolution Vault — {mesh_hires.number_of_vertices()} vertices",
           color='steelblue')
plt.tight_layout()
plt.show()

## Step 9: Color by Height

Height-mapped coloring shows the thrust distribution — useful for understanding the structural behavior.

In [ ]:
def plot_vault_colored(xyz, mesh, title="Vault", cmap='viridis', elev=30, azim=-60):
    """Plot vault with faces colored by average height."""
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    face_verts = []
    face_heights = []
    for face in mesh.faces():
        fv = mesh.face_vertices(face)
        polygon = [xyz[v] for v in fv]
        face_verts.append(polygon)
        face_heights.append(np.mean([xyz[v][2] for v in fv]))
    
    # Normalize heights for colormap
    heights = np.array(face_heights)
    norm = plt.Normalize(heights.min(), heights.max())
    colormap = plt.cm.get_cmap(cmap)
    face_colors = [colormap(norm(h)) for h in heights]
    
    poly = Poly3DCollection(face_verts, alpha=0.85)
    poly.set_facecolors(face_colors)
    poly.set_edgecolor('none')
    ax.add_collection3d(poly)
    
    ax.set_xlim(xyz[:, 0].min() - 0.5, xyz[:, 0].max() + 0.5)
    ax.set_ylim(xyz[:, 1].min() - 0.5, xyz[:, 1].max() + 0.5)
    ax.set_zlim(-0.5, xyz[:, 2].max() + 1.0)
    ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)'); ax.set_zlabel('Z (m)')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.view_init(elev=elev, azim=azim)
    
    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax, shrink=0.5, label='Height (m)')
    
    return fig, ax


plot_vault_colored(hr_xyz, mesh_hires, 
                   title='Vault — Height Map', cmap='plasma')
plt.tight_layout()
plt.show()

## What's Next

We've established the foundation:
- Created meshes from rectangular floor plans
- Used the Force Density Method to find compression-only vault shapes
- Explored how force density controls the height and shape
- Created directional (barrel) vaults with asymmetric force densities
- Visualized vaults with surface plots, cross-sections, and height maps

**Next notebook:** Deep dive into force density parameters — sweeps, gradients, and how they map to real structural behavior.

**Later:** The vault becomes discrete bricks. Each face in the mesh becomes a volumetric voxel. The smooth surface disappears — what remains is an assembly of blocks held together purely by compression. That's when it gets interesting.